In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt

In [70]:
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis\files\axis04.pdf'
tables=camelot.read_pdf(f,flavor='stream' ,pages = "all",edge_tol=500) 

In [71]:
df = pd.DataFrame; tmp = pd.DataFrame;

In [72]:
tables

<TableList n=6>

In [82]:
tmp = tables[3].df
tmp

,0,1,2,3,4,5,6,7
0,,,,,,,,CENTRALISED CTS MUM
1,20-11-2018,20-11-2018,NACH-DR- NEOGROWTH CREDIT PVT,,55834.00,DR,399813.23,MH
2,,,,,,,,RETAIL ASSET
3,20-11-2018,20-11-2018,BPR086102781710_EMI_20-11-2018,,36152.00,DR,363661.23,MANAGEMENT GR-RAM
4,20-11-2018,20-11-2018,NEFT/N324180169696088/LENDINGKART,,576.90,CR,364238.13,RTGS HUB
...,...,...,...,...,...,...,...,...
66,04-12-2018,04-12-2018,IMPS/P2A/833822353058/910020037211732/041218,,33505.90,CR,1197823.53,"POWAI, MUMBAI [MH]"
67,04-12-2018,04-12-2018,IMPS/P2A/833823365648/007863700000481YESB0000078,,33205.90,DR,1164617.63,"POWAI, MUMBAI [MH]"
68,,,,,,,,CENTRALISED CTS MUM
69,05-12-2018,05-12-2018,NACH-DR- EDELWEISSRETAILFINLT,,54981.00,DR,1109636.63,MH


In [5]:
tmp = tables[].df
try:
    idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
    tmp.columns=tmp.iloc[idx]
except:
    try:
        idx=[ c for c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
        tmp.columns=tmp.iloc[idx]
    except:
        print("\nAxis-Column headers missing")
# tmp.shape[1]
# tmp=pd.DataFrame(tmp.T.drop_duplicates().T).replace(r'^\s*$', np.nan, regex=True)

In [6]:
tmp.columns

Index(['Tran Date', 'Value Date', 'Transaction Particulars', 'Chq No',
       'Amount(INR)', 'DR/CR', 'Balance(INR)', 'Branch Name'],
      dtype='object', name=2)

In [7]:
if len(tmp.columns) == 8:
    print("true")

true


In [8]:
if (tmp.shape[1] == 6):
    tmp[6] = np.nan
    tmp = tmp[[0,6,1,2,3,4,5]]
    tmp.columns = range(tmp.shape[1])

In [9]:
tmp.head()

2,Tran Date,Value Date,Transaction Particulars,Chq No,Amount(INR),DR/CR,Balance(INR),Branch Name
0,PIN400076,,,,,,,Currency :INR
1,,,,Statement of Axis Account No :910020037211732 ...,,,,
2,Tran Date,Value Date,Transaction Particulars,Chq No,Amount(INR),DR/CR,Balance(INR),Branch Name
3,,,OPENING BALANCE,,,,309677.68,
4,01-11-2017,01-11-2017,NEFT/171101076GN00206/FLIPKART INTERNET P LTD ...,,1158.38,CR,310836.06,RTGS HUB


In [152]:
def came(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='1,2', edge_til=500)
    if len(tables) != 0:
        dt=pd.DataFrame(); tmp=pd.DataFrame()
        first_check = tables[0].df; second_check = tables[1].df;
        print(first_check.shape[1])
        print(second_check.shape[1])
        if (first_check.shape[1] > 5):
            try:
                idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                first_check.columns=first_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    first_check.columns=first_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            print("\nlength of first_check:",len(first_check.columns))
            if len(first_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(first_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df   
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
            elif len(first_check.columns) > 8:
                print("Column length greater than 8")
                for i in range(len(tables2)):
                    print(tmp.shape[1])
                    if (tmp.shape[1] > 8):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                            except:
                                print("\nAxis-Column headers missing"); pass
                        tmp.drop("",axis = 1,inplace = True)
                        tmp.columns = [0,1,2,3,4,5,6,7]
                        dt=pd.concat([dt,tmp]).drop_duplicates().reset_index(drop=True)
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                            except:
                                print("\nAxis-Column headers missing")
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
                        tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
                        if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                            tmp.columns = ["A","B","C","D","E","F"]
                            tmp.reset_index(drop = True, inplace=True)
                            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                            result = pd.concat([tmp1, tmp], axis=1, sort=False)
                            result.drop('A' ,axis=1, inplace=True)
                            result["Y"] = result["Z1"]
                            for i,j in enumerate(result["Y"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                else:
                                    pass
                                    
                            for i,j in enumerate(result["Z1"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Z1'] = ""
                                    pass
                                else:
                                    result.at[i, 'Z1'] = ""
                                    pass 
                            result.fillna(value=pd.np.nan, inplace=True)
                            result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                            result.drop(['Z3','Y'] ,axis=1, inplace=True)
                            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                            result.columns = [0,1,2,3,4,5,6,7]
                            dt=pd.concat([dt,result]).drop_duplicates().reset_index(drop=True)
                        else:
                            print("\nIn without box pdf another Structure on page:",i); pass

        elif(second_check.shape[1] > 5):
            try:
                idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                second_check.columns=second_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    second_check.columns=second_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            if len(second_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(second_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

    else:
        print("\nLength of table is 0"); pass   
    
    return dt

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis\files\axis02.pdf'
output = came(f)
output.to_csv("output.csv", index=0)
output

8
7

length of first_check: 8
Column length is 8

Diiferent structure on page: 30


,0,1,2,3,4,5,6,7
0,PIN400076,NaN,NaN,NaN,NaN,NaN,NaN,Currency :INR
1,NaN,NaN,NaN,Statement of Axis Account No :910020037211732 ...,NaN,NaN,NaN,NaN
2,Tran Date,Value Date,Transaction Particulars,Chq No,Amount(INR),DR/CR,Balance(INR),Branch Name
3,NaN,NaN,OPENING BALANCE,NaN,NaN,NaN,309677.68,NaN
4,01-11-2017,01-11-2017,NEFT/171101076GN00206/FLIPKART INTERNET P LTD ...,NaN,1158.38,CR,310836.06,RTGS HUB
...,...,...,...,...,...,...,...,...
1673,31-10-2018,31-10-2018,IMPS/P2A/830412412196/919768667157/ATOMNETBANKIN,NaN,145000.00,CR,555297.47,"POWAI, MUMBAI [MH]"
1674,31-10-2018,31-10-2018,NEFT/N304180163207887/LENDINGKART,NaN,6730.70,CR,562028.17,DATA CENTRE-CHEMBUR
1675,31-10-2018,31-10-2018,NEFT/N304180163208334/LENDINGKART,NaN,178.89,CR,562207.06,RTGS HUB
1676,NaN,NaN,TRANSACTION TOTAL DR/CR,NaN,47314531.11/47567060.49,NaN,NaN,NaN


# ******************************

In [154]:
def came(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='1,2', edge_til=500)
    if len(tables) != 0:
        dt=pd.DataFrame(); tmp=pd.DataFrame()
        first_check = tables[0].df; second_check = tables[1].df;
        print(first_check.shape[1])
        print(second_check.shape[1])
        if (first_check.shape[1] > 5):
            try:
                idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                first_check.columns=first_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    first_check.columns=first_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            print("\nlength of first_check:",len(first_check.columns))
            if len(first_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(first_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df   
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
            elif len(first_check.columns) > 8:
                print("Column length greater than 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df 
#                     print(tmp.shape[1])
                    if (tmp.shape[1] > 8):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                            except:
                                print("\nAxis-Column headers missing"); pass
                        tmp.drop("",axis = 1,inplace = True)
                        tmp.columns = [0,1,2,3,4,5,6,7]
                        dt=pd.concat([dt,tmp]).drop_duplicates().reset_index(drop=True)
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                            except:
                                print("\nAxis-Column headers missing")
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
                        tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
                        if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                            tmp.columns = ["A","B","C","D","E","F"]
                            tmp.reset_index(drop = True, inplace=True)
                            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                            result = pd.concat([tmp1, tmp], axis=1, sort=False)
                            result.drop('A' ,axis=1, inplace=True)
                            result["Y"] = result["Z1"]
                            for i,j in enumerate(result["Y"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                else:
                                    pass
                                    
                            for i,j in enumerate(result["Z1"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Z1'] = ""
                                    pass
                                else:
                                    result.at[i, 'Z1'] = ""
                                    pass 
                            result.fillna(value=pd.np.nan, inplace=True)
                            result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                            result.drop(['Z3','Y'] ,axis=1, inplace=True)
                            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                            result.columns = [0,1,2,3,4,5,6,7]
                            dt=pd.concat([dt,result]).drop_duplicates().reset_index(drop=True)
                        else:
                            print("\nIn without box pdf another Structure on page:",i); pass

        elif(second_check.shape[1] > 5):
            try:
                idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                second_check.columns=second_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    second_check.columns=second_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            if len(second_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(second_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

    else:
        print("\nLength of table is 0"); pass   
    
    return dt

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis\files\axis03.pdf'
output = came(f)
output.to_csv("output02.csv", index=0)
output

9
8

length of first_check: 9
Column length greater than 8
9
8
6


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


8
8
6

Axis-Column headers missing

In without box pdf another Structure on page: 5
8
8
8
8
8
8
8
8
8
8
5
4
4


,0,1,2,3,4,5,6,7
0,Tran Date,Value Date,Transaction Particulars,Chq No.,Amount,DR|CR,Balance,Branch Name
1,,,OPENING BALANCE :,,,,25184.55,
2,,,,,,,,ATM
3,03-12-2018,03-12-2018,EDC/00049-00051-/M037011032984700,,5053.95,CR,30238.50,RECONCILATION
4,,,,,,,,CENTRE
...,...,...,...,...,...,...,...,...
272,10-11-2019,10-11-2019,ECOM PUR/One97 Communi/Noida/101119/19:04,NaN,334.98,DR,4748.06,NaN
273,15-11-2019,15-11-2019,IMPS/P2A/931916091620/RANJIT/HDFCBAN/X080290/,NaN,202.95,DR,4545.11,NaN
274,16-11-2019,16-11-2019,NEFT/N320190983426385/INDU BHARGAVA/Payment,NaN,3300.00,CR,7845.11,RTGS HUB
275,20-11-2019,20-11-2019,ECOM PUR/One97 Communi/Noida/201119/15:01,NaN,60.00,DR,7785.11,NaN


In [147]:
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis\files\axis02.pdf'
tables = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
if len(tables) != 0:
    df=pd.DataFrame(); tmp=pd.DataFrame();
    first_check = tables[0].df; second_check = tables[1].df;

    if (first_check.shape[1] < 5):
        print("first_check")

    elif(second_check.shape[1] > 5):
        print("second_check")

second_check


In [148]:
tables

<TableList n=31>

In [144]:
tables[8].df

,0,1,2,3,4,5,6
0,01-02-2018,01-02-2018,IMPS/P2A/803208798380/194602000000357IOBA0001946,200017.70,DR,2305171.63,"POWAI, MUMBAI [MH]"
1,01-02-2018,01-02-2018,IMPS/P2A/803208798705/194602000000197IOBA0001946,200017.70,DR,2105153.93,"POWAI, MUMBAI [MH]"
2,01-02-2018,01-02-2018,IMPS/P2A/803208888541/919768667157/to aaditya c,200000.00,CR,2305153.93,"POWAI, MUMBAI [MH]"
3,01-02-2018,01-02-2018,IMPS/P2A/803208800533/194602000000197IOBA0001946,165017.70,DR,2140136.23,"POWAI, MUMBAI [MH]"
4,01-02-2018,01-02-2018,IMPS/P2A/803208800774/194602000000357IOBA0001946,135017.70,DR,2005118.53,"POWAI, MUMBAI [MH]"
...,...,...,...,...,...,...,...
57,,,,,,,CENTRALISED CTS MUM
58,08-02-2018,08-02-2018,NACH-DR- INTELLECASH MICROFIN,102083.00,DR,1467036.01,MH
59,08-02-2018,08-02-2018,IMPS/P2A/803914490628/910000000000/aaditya,72000.00,CR,1539036.01,"POWAI, MUMBAI [MH]"
60,,,,,,,CENTRALISED COLL &


# 19 feb 20

In [61]:
def came(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='1,2', edge_tol=500)
    if len(tables) != 0:
        dt=pd.DataFrame(); tmp=pd.DataFrame()
        first_check = tables[0].df; second_check = tables[1].df;
        # print(first_check.shape[1])
        # print(second_check.shape[1])
        if (first_check.shape[1] > 5):
            try:
                idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                first_check.columns=first_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    first_check.columns=first_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_tol=500)
            print("\nlength of first_check:",len(first_check.columns))
            if len(first_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(first_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df   
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
            elif len(first_check.columns) > 8:
                print("Column length greater than 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df 
                    if (tmp.shape[1] > 8):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                            except:
                                print("\nAxis-Column headers missing"); pass
                        tmp.drop("",axis = 1,inplace = True)
                        tmp.columns = [0,1,2,3,4,5,6,7]
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        # print("\n",tmp)
                        dt=pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        # print("\n",tmp)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        # print("\n",tmp)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                            except:
                                print("\nAxis-Column headers missing")
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
                        tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
                        if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                            tmp.columns = ["A","B","C","D","E","F"]
                            tmp.reset_index(drop = True, inplace=True)
                            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                            result = pd.concat([tmp1, tmp], axis=1, sort=False)
                            result.drop('A' ,axis=1, inplace=True)
                            result["Y"] = result["Z1"]
                            for i,j in enumerate(result["Y"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                else:
                                    pass
                                    
                            for i,j in enumerate(result["Z1"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Z1'] = ""
                                    pass
                                else:
                                    result.at[i, 'Z1'] = ""
                                    pass 
                            result.fillna(value=pd.np.nan, inplace=True)
                            result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                            result.drop(['Z3','Y'] ,axis=1, inplace=True)
                            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                            result.columns = [0,1,2,3,4,5,6,7]
                            result=pd.DataFrame(result).replace(r'^\s*$', np.nan, regex=True)
                            # print("\n",result)
                            dt=pd.concat([dt,result]).reset_index(drop=True)
                        else:
                            print("\nIn without box pdf another Structure on page:",i); pass

        elif(second_check.shape[1] > 5):
            try:
                idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                second_check.columns=second_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    second_check.columns=second_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            if len(second_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(second_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    #print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

    else:
        print("\nLength of table is 0"); pass   
    
    return dt

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis_new\files\axis03.pdf'
output = came(f)
output.to_csv("output03.csv", index=0)
# output


length of first_check: 9
Column length greater than 8


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [64]:
def came(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='1,2', edge_tol=500)
    if len(tables) != 0:
        dt=pd.DataFrame(); tmp=pd.DataFrame()
        first_check = tables[0].df; second_check = tables[1].df;
        # print(first_check.shape[1])
        # print(second_check.shape[1])
        if (first_check.shape[1] > 5):
            try:
                idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                first_check.columns=first_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    first_check.columns=first_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_tol=500)
            print("\nlength of first_check:",len(first_check.columns))
            if len(first_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(first_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df   
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
            elif len(first_check.columns) > 8:
                print("Column length greater than 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df 
                    if (tmp.shape[1] > 8):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                            except:
                                print("\nAxis-Column headers missing"); pass
                        tmp.drop("",axis = 1,inplace = True)
                        tmp.columns = [0,1,2,3,4,5,6,7]
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        # print("\n",tmp)
                        dt=pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        # print("\n",tmp)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        # print("\n",tmp)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                            except:
                                print("\nAxis-Column headers missing")
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
                        tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
                        if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                            tmp.columns = ["A","B","C","D","E","F"]
                            tmp.reset_index(drop = True, inplace=True)
                            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                            result = pd.concat([tmp1, tmp], axis=1, sort=False)
                            result.drop('A' ,axis=1, inplace=True)
                            result["Y"] = result["Z1"]
                            for i,j in enumerate(result["Y"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                else:
                                    pass
                                    
                            for i,j in enumerate(result["Z1"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Z1'] = ""
                                    pass
                                else:
                                    result.at[i, 'Z1'] = ""
                                    pass 
                            result.fillna(value=pd.np.nan, inplace=True)
                            result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                            result.drop(['Z3','Y'] ,axis=1, inplace=True)
                            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                            result.columns = [0,1,2,3,4,5,6,7]
                            result=pd.DataFrame(result).replace(r'^\s*$', np.nan, regex=True)
                            # print("\n",result)
                            dt=pd.concat([dt,result]).reset_index(drop=True)
                        else:
                            print("\nIn without box pdf another Structure on page:",i); pass

        elif(second_check.shape[1] > 5):
            try:
                idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                second_check.columns=second_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    second_check.columns=second_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            if len(second_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(second_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    #print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

    else:
        print("\nLength of table is 0"); pass   
    
    return dt

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis_new\files\axis02.pdf'
output2 = came(f)
# output2.to_csv("output03.csv", index=0)
# output

Column length is 8

Diiferent structure on page: 30


In [374]:
def came(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='1,2', edge_tol=500)
    if len(tables) != 0:
        dt=pd.DataFrame(); tmp=pd.DataFrame()
        first_check = tables[0].df; second_check = tables[1].df;
        print(first_check.shape[1])
        print(second_check.shape[1])
        if (first_check.shape[1] > 5):
            try:
                idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                first_check.columns=first_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    first_check.columns=first_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_tol=500)
            print("\nlength of first_check:",len(first_check.columns))
            if len(first_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(first_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df   
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
            elif len(first_check.columns) > 8:
                print("Column length greater than 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df 
                    if (tmp.shape[1] > 8):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                            except:
                                print("\nAxis-Column headers missing"); pass
                        tmp.drop("",axis = 1,inplace = True)
                        tmp.columns = [0,1,2,3,4,5,6,7]
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        # print("\n",tmp)
                        dt=pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        # print("\n",tmp)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        # print("\n",tmp)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                            except:
                                print("\nAxis-Column headers missing")
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
                        tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
                        if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                            tmp.columns = ["A","B","C","D","E","F"]
                            tmp.reset_index(drop = True, inplace=True)
                            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                            result = pd.concat([tmp1, tmp], axis=1, sort=False)
                            result.drop('A' ,axis=1, inplace=True)
                            result["Y"] = result["Z1"]
                            for i,j in enumerate(result["Y"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                else:
                                    pass
                                    
                            for i,j in enumerate(result["Z1"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Z1'] = ""
                                    pass
                                else:
                                    result.at[i, 'Z1'] = ""
                                    pass 
                            result.fillna(value=pd.np.nan, inplace=True)
                            result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                            result.drop(['Z3','Y'] ,axis=1, inplace=True)
                            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                            result.columns = [0,1,2,3,4,5,6,7]
                            result=pd.DataFrame(result).replace(r'^\s*$', np.nan, regex=True)
                            # print("\n",result)
                            dt=pd.concat([dt,result]).reset_index(drop=True)
                        else:
                            print("\nIn without box pdf another Structure on page:",i); pass

        elif(second_check.shape[1] > 5):
            try:
                idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                second_check.columns=second_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    second_check.columns=second_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_tol=500)
            print("********************************************************************")
            print(len(second_check.columns))
            print("********************************************************************")
            if len(second_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(second_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    #print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
                    
            elif len(second_check.columns) == 6:
                print("Column length is 6")
                for i in range(len(tables2)):
                    print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 6):
                        print("\n",tmp)
                        print("\nDiiferent structure on page:",i); pass;
                        print("\n")
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\n",tmp)
                        print("\nAnother Structure on page:",i); pass
                        print("\n")

    else:
        print("\nLength of table is 0"); pass   
    
    return dt

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis_new\files\indus.pdf'
output3 = came(f)
output3.to_csv("output03.csv", index=0)
# output

4
6
********************************************************************
6
********************************************************************
Column length is 6

 0 : 0

                                                     0  \
0                                       IndusInd Bank   
1                                                       
2                                                       
3                NITTHALEE INFOTECH PROGRAMMING LAMIA   
4                                                       
5                                                       
6                                                       
7   7 AND 11 KUMUND VIHAR SECTOR-3 PRATAP NAGARNEA...   
8                                                       
9                                     JAIPUR - 302029   
10                                   RAJASTHAN, INDIA   
11                                                      
12                                                      
13                           

In [349]:
df = output3

In [350]:
try:
    idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
    df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
    print("1st attempt")
except:
    try:
        idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
        df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
    except:
        print("\nAxis-Column headers missing"); pass

1st attempt


In [351]:
df.head()

1,Tran Date,Chq No,Particulars,Debit,Credit,Balance,Init.
0,NaN,NaN,NaN,NaN,NaN,NaN,Br
1,NaN,NaN,OPENING BALANCE,NaN,NaN,1880.22,NaN
2,NaN,NaN,IMPS/P2A/924413019791/CRYSTALV/THEGRE,NaN,NaN,NaN,NaN
3,01-09-2019,NaN,AT/,NaN,2000.00,3880.22,1572
4,NaN,NaN,MOB/QYW6RJS14956/Ratnakar Bank Credit,NaN,NaN,NaN,NaN


In [352]:
df.to_csv("output03.csv", index=0)

In [353]:
try:
    df = df.drop(["Init."], axis=1)
except:
    try:
        df = df.drop(["Branch Name"], axis=1)
    except:pass

In [354]:
df.head()

1,Tran Date,Chq No,Particulars,Debit,Credit,Balance
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,OPENING BALANCE,NaN,NaN,1880.22
2,NaN,NaN,IMPS/P2A/924413019791/CRYSTALV/THEGRE,NaN,NaN,NaN
3,01-09-2019,NaN,AT/,NaN,2000.00,3880.22
4,NaN,NaN,MOB/QYW6RJS14956/Ratnakar Bank Credit,NaN,NaN,NaN


In [355]:
df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance|particulars|transaction').any(), axis=1) ==True].index)]
# df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)
df.head()

1,Tran Date,Chq No,Particulars,Debit,Credit,Balance
0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,IMPS/P2A/924413019791/CRYSTALV/THEGRE,NaN,NaN,NaN
3,01-09-2019,NaN,AT/,NaN,2000.00,3880.22
4,NaN,NaN,MOB/QYW6RJS14956/Ratnakar Bank Credit,NaN,NaN,NaN
5,01-09-2019,NaN,Card,2000.00,NaN,1880.22


In [356]:
df.to_csv("output04.csv", index=0)

In [357]:
try:
    narr=[n for n in df.columns if "PARTICULARS" in str(n).upper()][0]
except:
    try:
        narr=[n for n in df.columns if "NARRATION" in str(n).upper()][0]
    except:
        try:
            narr=[n for n in df.columns if "DESCRIPTION" in str(n).upper()][0] 
        except:
            pass

In [358]:
narr

'Particulars'

In [359]:
df.to_csv("output05.csv", index=0)

In [360]:
df.head()

1,Tran Date,Chq No,Particulars,Debit,Credit,Balance
0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,IMPS/P2A/924413019791/CRYSTALV/THEGRE,NaN,NaN,NaN
3,01-09-2019,NaN,AT/,NaN,2000.00,3880.22
4,NaN,NaN,MOB/QYW6RJS14956/Ratnakar Bank Credit,NaN,NaN,NaN
5,01-09-2019,NaN,Card,2000.00,NaN,1880.22


In [361]:
# df.columns.str.lower().str.contains('inr|debit|credit').any()

In [362]:
# list1 = list(df.columns.values)

In [365]:
print(list(df.columns.values))

['Tran Date', 'Chq No', 'Particulars', 'Debit', 'Credit', 'Balance']


In [366]:
list1 = ['Tran Date','Chq No','Particulars','Debit','Credit','Balance']
list2 = ['Tran Date','Value Date','Transaction Particulars','Chq No','Amount(INR)','DR/CR','Balance(INR)']
list3 = ['Tran Date','Value Date','Transaction Particulars','Chq No.','Amount','DR|CR','Balance']

if list(df.columns.values) == list1:
    print("list1")
elif list(df.columns.values) == list2:
    print("list2")
elif list(df.columns.values) == list3:
    print("list3")

list1


In [339]:
list1 = ['Tran Date','Chq No','Particulars','Debit','Credit','Balance']
list2 = ['Tran Date','Value Date','Transaction Particulars','Chq No','Amount(INR)','DR/CR','Balance(INR)']
list3 = ['Tran Date','Value Date','Transaction Particulars','Chq No.','Amount','DR|CR','Balance']

def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

def isnan(value):
        try:
            import math
            return math.isnan(float(value))
        except:
            return False

def backfill(df):
    df = df[pd.notnull(df[narr])]
    df.iloc[:,-1]=df.iloc[:,-1].fillna(df.iloc[:,-2])
    
    try:
        bal=[c for c in df.columns if "BALANCE" in str(c).upper()][0]
        df[bal]=df[bal].apply(lambda x: abc(x))
    except:
        print("\nBalance Column Missing")
        
    df["flag"]=df.iloc[:,-1].shift(1)
    df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]

    df=df.T.drop_duplicates().T
    df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)
    df["flag"]=df.iloc[:,0].astype(str)+df[bal].astype(str)
    df["flag2"]=np.arange(len(df))

    df.loc[df[["flag"]].duplicated(keep=False), 'flag'] = df["flag"]+df["flag2"].astype(str)
    df['flag']=df['flag'].apply(lambda row : np.nan if 'nannan' in row else row).fillna(method='bfill')

    df[narr]=df.groupby('flag')[narr].transform(lambda x:''.join(x))
    df=df.drop_duplicates(['flag'],keep='last').iloc[0:,:-2].reset_index(drop=True)
    df.to_csv("parsed_completed.csv",index=0)
    print("parsed_completed")

if list(df.columns.values) == list1 :
    print("list1")
    backfill(df)
    
elif list(df.columns.values) == list2 :
    print("list2")
    backfill(df)

elif list(df.columns.values) == list3:
    print("list3")   
    
    try:
        bal=[c for c in df.columns if "BALANCE" in str(c).upper()][0]
        df[bal]=df[bal].apply(lambda x: abc(x))
    except:
        print("\nBalance Column Missing")

    try:    
        df.replace(r'^\s*$', np.nan, regex=True, inplace = True)
        #type(df["Transaction Particulars"][15])
    except:pass

    try:
        df.dropna(axis=0, how='all',inplace=True)
    except:pass

    try:
        df.reset_index(drop=True,inplace=True)
    except:pass

    try:
        for j,i in enumerate(df["Transaction Particulars"]):
            if isnan(i) == True:
                df[narr][j] = df[narr][j-1]+df[narr][j+1]
            else:
                pass
    except:pass

    try:
        df.dropna(subset=['Tran Date'], inplace = True)
    except:
        try:
            df.dropna(subset=['Value Date'], inplace = True)
        except:pass

    df.to_csv("parsed_completed.csv",index=0)
    print("parsed_completed")
    
else:
    print("true")

list2


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


parsed_completed


In [30]:
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis_new\files\boi.pdf'
tables2 = camelot.read_pdf(f,flavor='stream' ,pages = "all", edge_tol=500)
dt=pd.DataFrame(); tmp=pd.DataFrame()

for i in range(len(tables2)):
    tmp = tables2[i].df
    tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
    dt = pd.concat([dt,tmp]).reset_index(drop=True)
    
dt.to_csv("output.csv",index=0)

In [31]:
tables2

<TableList n=13>

In [17]:
tables2[31].df

,0,1,2,3,4,5,6,7
0,DATE,,PARTICULARS,CHQ.NO.,WITHDRAWALS,DEPOSITS,BALANCE,
1,,NFS CASH TXNINHP,,,,,,
2,01-DEC-19,,,,5000.00 Dr,,,511.80 Cr
3,,PETROLPUMPGOPALPURAJAIPUR,,,,,,
4,,UPI933545657434DRPaytPYTMadd-,,,,,,
5,01-DEC-19,,,,130.00 Dr,,,381.80 Cr
6,,moneypaytmOid9,,,,,,
7,,UPI933545683487DRPaytPYTMadd-,,,,,,
8,01-DEC-19,,,,185.00 Dr,,,196.80 Cr
9,,moneypaytmOid9,,,,,,


In [28]:
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis_new\files\indus.pdf'
tables=tabula.read_pdf(f,
                      pages='all',
                      multiple_tables=True,
                      pandas_options={'header':None})
df = pd.DataFrame()
df = pd.concat([c for c in tables]).drop_duplicates()


In [29]:
df.to_csv("output.csv", index=0)